In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import pandas as pd
import plotly.express as p

# Load data
data = pd.read_csv(
    '/Users/saikatbasu/Documents/PersonalInformation_Oct2022/IBMDataScience-Coursera/AppliedDataScience-10of12/Module3-InteractiveVisualAnalytics/spacex_launch_dash.csv'
)
data.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [2]:
# Initialize the app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'}] +
                [{'label': site, 'value': site} for site in data['Launch Site'].unique()],
        value='ALL',
        placeholder='Select Launch Site'
    ),
    
    dcc.Graph(id='output-container'),
    
    dcc.RangeSlider(id='payload-slider',min=0, max=10000, step=1000, value=[data['Payload Mass (kg)'].min(), data['Payload Mass (kg)'].max()]),

    dcc.Graph(id='success-payload-scatter')
])

In [6]:
# Callback for dropdown list of Launch Sites

@app.callback(
    Output('output-container', 'figure'),
    Input('site-dropdown', 'value')
)

def update_graph(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(data, names='Launch Site', values='class', title='Total Success Launches by Site')
    else:
        filtered = data[data['Launch Site'] == selected_site]
        fig = px.pie(filtered, names = 'class', title=f('Launch Outcomes for {selected_site}'))
    return fig

In [7]:
# Callback for slide-ranger for choosing Payload mass

@app.callback(
    Output('success-payload-scatter', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)

def update_scatter(selected_site, payload_range):
    low, high = payload_range
    mask = (data['Payload Mass (kg)'] >= low) & (data['Payload Mass (kg)'] <= high)
    filtered = data[mask]

    if selected_site != 'ALL':
        filtered = filtered[filtered['Launch Site'] == selected_site]

    fig = px.scatter(
        filtered, x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title='Payload vs. Outcome'
    )
    return fig

In [9]:
# Finally launcing the app in debug mode

# Run
if __name__ == '__main__':
    app.run(debug=True)

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1